### Setup

In [ ]:
# %cd ~

In [ ]:
# ~/.cache/huggingface

In [1]:
import wandb
import os
import time

wandb.login()

wandb: Currently logged in as: matt24. Use `wandb login --relogin` to force relogin


True

In [2]:
import torch

torch.cuda.empty_cache()
print(torch.cuda.memory_allocated() / (1024 ** 3))
print(torch.cuda.memory_reserved() / (1024 ** 3))

0.0
0.0


In [3]:
from diffusers import StableDiffusionPipeline
import torch
import random
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"


def image_grid(imgs, rows, cols, size=None):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))

    if size:
        return grid.resize((cols * size, rows * size))
    else:   
        return grid

import os
import math

# define function to get size of directory in bytes
def get_dir_size(path):
    total_size = os.path.getsize(path)
    if os.path.isdir(path):
        for item in os.listdir(path):
            item_path = os.path.join(path, item)
            if os.path.isfile(item_path):
                total_size += os.path.getsize(item_path)
            elif os.path.isdir(item_path):
                total_size += get_dir_size(item_path)
    return total_size

In [4]:
import diffusers
from diffusers import StableDiffusionPipeline, PNDMScheduler, EulerDiscreteScheduler, DDIMScheduler
import torch

schedulers = {
    "pndm": PNDMScheduler,
    "euler": EulerDiscreteScheduler,
    "ddim": DDIMScheduler
}

### Deliberate

In [5]:
import diffusers
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
from transformers import CLIPImageProcessor
import torch

PROJECT_NAME = "sd-profiler"

model_id = "XpucT/Deliberate"
scheduler = schedulers["pndm"].from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
pipe = pipe.to(device)

print(f"Scheduler: {pipe.scheduler.config['_class_name']}")

torch.cuda.empty_cache()
print(torch.cuda.memory_allocated() / (1024 ** 3))
print(torch.cuda.memory_reserved() / (1024 ** 3))

# model_path = "./.cache/huggingface/hub/models--" + model_id.replace("/", "--")

# # get size of directory in bytes
# dir_size = get_dir_size(model_path)

# # convert to GB
# dir_size_gb = dir_size / (1024**3)

# print(f"Directory size {math.ceil(dir_size_gb)} GB")

safety_checker/model.safetensors not found


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


Scheduler: PNDMScheduler
2.588134765625
2.6015625


In [6]:
prompts = [
    "The most beautiful cat nowadays can be described as a Bengal cat",
    "The most beautiful cat nowadays can be described as a Bengal cat. They have short, smooth and shiny fur with various colors, from orange, brown to gray and black",
    "The most beautiful cat nowadays can be described as a Bengal cat. They have short, smooth and shiny fur with various colors, from orange, brown to gray and black. The highlight of their appearance is the bold black stripes on their fur, creating a wild and charming look. They have big, bright eyes with green or yellow color"
]

In [8]:
n = 1
for inference_step in [20, 40, 60]:
    for sched in ["pndm", "ddim"]:
        for prompt in prompts:
            model_id = "XpucT/Deliberate"
            scheduler = schedulers[sched].from_pretrained(model_id, subfolder="scheduler")
            pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
            pipe = pipe.to(device)
            
            config = {}
            
            config["prompt"] = "The most beautiful cat nowadays can be described as a Bengal cat. They have short, smooth and shiny fur with various colors, from orange, brown to gray and black. The highlight of their appearance is the bold black stripes on their fur, creating a wild and charming look. They have big, bright eyes with green or yellow color, giving them a sharp and lively look"
            config["word_count"] = len(config["prompt"].split())
            config["token_count"] = len(pipe.tokenizer(config["prompt"])["input_ids"])
            
            config["num_inference_steps"] = inference_step
            config["guidance_scale"] = 7.5
            config["num_images_per_prompt"] = 1
            
            config["cpu_count"] = os.cpu_count()
            config["gpu_count"] = torch.cuda.device_count()
            config["gpu_type"] = torch.cuda.get_device_name(0)
            config["cuda_memory_allocated (GB)"] = round(torch.cuda.memory_allocated() / (1024 ** 3), 2)
            
            for k, v in pipe.config.items():
                if k == "_name_or_path":
                    k = "model_id"
                
                if isinstance(v, tuple):
                    config[k] = v[1]
                else:
                    config[k] = v
            
            
            samples = 3
            seeds = []
            images = []
            
            for i in range(samples):
                wandb.init(project=PROJECT_NAME, config=config)
                
                seed = random.randint(0, 1e6)
                generator = torch.Generator(device).manual_seed(seed)
                
                torch.cuda.empty_cache()
                start_cuda_malloc = torch.cuda.memory_allocated()
                start_cuda_mres = torch.cuda.memory_reserved()
                start_time = time.time()
                
                image = pipe(
                    prompt = config["prompt"],
                    num_inference_steps = config["num_inference_steps"],
                    guidance_scale = config["guidance_scale"],
                    num_images_per_prompt = config["num_images_per_prompt"],
                    generator=generator
                ).images[0]
                
                end_time = time.time()
                end_cuda_malloc = torch.cuda.memory_allocated()
                end_cuda_mres = torch.cuda.memory_reserved()
                elapsed_time = end_time - start_time
                cuda_malloc_usage = (end_cuda_malloc - start_cuda_malloc) / (1024 **3)
                cuda_mres_usage = (end_cuda_mres - start_cuda_mres) / (1024 ** 3)
                
                wandb.config.update({
                    "seed": seed,
                    "elapsed_time (seconds)": round(elapsed_time, 2),
                    "cuda_inference_usage (GB)": round(cuda_mres_usage, 2)    
                })
                
                wandb.finish()
                
                print(f"iteration: #{n}")
                print(f"Word count: {len(config['prompt'].split())}")
                print(f"Token count: {len(pipe.tokenizer(config['prompt'])['input_ids'])}")
                print(f"Inference Steps: {inference_step}")
                print(f"Scheduler: {scheduler}")
                
                print(f"Elapsed time: {round(elapsed_time, 2)} seconds")
                print(f"CUDA Model size: {round(start_cuda_malloc / (1024 ** 3), 2)} GB")
                print(f"CUDA Memory Allocated: {round(cuda_malloc_usage, 2)} GB")
                print(f"CUDA Memory Inference: {round(cuda_mres_usage, 2)} GB")

                n += 1
    
    #     images.append(image)
    #     seeds.append(seed)
    
    # print(seeds)
    # image_grid(images, 1, samples, 256)

safety_checker/model.safetensors not found


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


  0%|          | 0/20 [00:00<?, ?it/s]

iteration: #1
Word count: 65
Token count: 76
Inference Steps: 20
Scheduler: PNDMScheduler {
  "_class_name": "PNDMScheduler",
  "_diffusers_version": "0.19.3",
  "beta_end": 0.012,
  "beta_schedule": "scaled_linear",
  "beta_start": 0.00085,
  "clip_sample": false,
  "num_train_timesteps": 1000,
  "prediction_type": "epsilon",
  "set_alpha_to_one": false,
  "skip_prk_steps": true,
  "steps_offset": 1,
  "timestep_spacing": "leading",
  "trained_betas": null
}

Elapsed time: 1.87 seconds
CUDA Model size: 2.6 GB
CUDA Memory Allocated: 0.0 GB
CUDA Memory Inference: 0.98 GB


...

iteration: #52
Word count: 65
Token count: 76
Inference Steps: 60
Scheduler: DDIMScheduler {
  "_class_name": "DDIMScheduler",
  "_diffusers_version": "0.19.3",
  "beta_end": 0.012,
  "beta_schedule": "scaled_linear",
  "beta_start": 0.00085,
  "clip_sample": false,
  "clip_sample_range": 1.0,
  "dynamic_thresholding_ratio": 0.995,
  "num_train_timesteps": 1000,
  "prediction_type": "epsilon",
  "rescale_betas_zero_snr": false,
  "sample_max_value": 1.0,
  "set_alpha_to_one": false,
  "skip_prk_steps": true,
  "steps_offset": 1,
  "thresholding": false,
  "timestep_spacing": "leading",
  "trained_betas": null
}

Elapsed time: 4.89 seconds
CUDA Model size: 2.6 GB
CUDA Memory Allocated: 0.0 GB
CUDA Memory Inference: 0.98 GB


  0%|          | 0/60 [00:00<?, ?it/s]

iteration: #53
Word count: 65
Token count: 76
Inference Steps: 60
Scheduler: DDIMScheduler {
  "_class_name": "DDIMScheduler",
  "_diffusers_version": "0.19.3",
  "beta_end": 0.012,
  "beta_schedule": "scaled_linear",
  "beta_start": 0.00085,
  "clip_sample": false,
  "clip_sample_range": 1.0,
  "dynamic_thresholding_ratio": 0.995,
  "num_train_timesteps": 1000,
  "prediction_type": "epsilon",
  "rescale_betas_zero_snr": false,
  "sample_max_value": 1.0,
  "set_alpha_to_one": false,
  "skip_prk_steps": true,
  "steps_offset": 1,
  "thresholding": false,
  "timestep_spacing": "leading",
  "trained_betas": null
}

Elapsed time: 4.85 seconds
CUDA Model size: 2.6 GB
CUDA Memory Allocated: 0.0 GB
CUDA Memory Inference: 0.98 GB


  0%|          | 0/60 [00:00<?, ?it/s]

iteration: #54
Word count: 65
Token count: 76
Inference Steps: 60
Scheduler: DDIMScheduler {
  "_class_name": "DDIMScheduler",
  "_diffusers_version": "0.19.3",
  "beta_end": 0.012,
  "beta_schedule": "scaled_linear",
  "beta_start": 0.00085,
  "clip_sample": false,
  "clip_sample_range": 1.0,
  "dynamic_thresholding_ratio": 0.995,
  "num_train_timesteps": 1000,
  "prediction_type": "epsilon",
  "rescale_betas_zero_snr": false,
  "sample_max_value": 1.0,
  "set_alpha_to_one": false,
  "skip_prk_steps": true,
  "steps_offset": 1,
  "thresholding": false,
  "timestep_spacing": "leading",
  "trained_betas": null
}

Elapsed time: 4.9 seconds
CUDA Model size: 2.6 GB
CUDA Memory Allocated: 0.0 GB
CUDA Memory Inference: 0.98 GB


In [ ]:
image_grid(images, 1, samples, 256)

In [ ]:
# %%time

import time
 

samples = 4
seeds = []
images = []

for i in range(samples):
    wandb.init(project=PROJECT_NAME, config=config) 
    seed = random.randint(0, 1e6)
    generator = torch.Generator(device).manual_seed(seed)
    torch.cuda.empty_cache()
    start_cuda_malloc = torch.cuda.memory_allocated()
    start_cuda_mres = torch.cuda.memory_reserved()
    start_time = time.time()
    image = pipe(
        prompt = config["prompt"],
        num_inference_steps = config["num_inference_steps"],
        guidance_scale = config["guidance_scale"],
        num_images_per_prompt = config["num_images_per_prompt"],
        generator=generator
    ).images[0]
    end_time = time.time()
    end_cuda_malloc = torch.cuda.memory_allocated()
    end_cuda_mres = torch.cuda.memory_reserved()
    elapsed_time = end_time - start_time
    cuda_malloc_usage = (end_cuda_malloc - start_cuda_malloc) / (1024 **3)
    cuda_mres_usage = (end_cuda_mres - start_cuda_mres) / (1024 ** 3)
    
    wandb.config.update({
        "seed": seed,
        "elapsed_time (seconds)": round(elapsed_time, 2),
        "cuda_inference_usage (GB)": round(cuda_mres_usage, 2)    
    })
    
    wandb.finish()

    print(f"Word count: {len(config['prompt'].split())}")
    print(f"Token count: {len(pipe.tokenizer(config['prompt'])['input_ids'])}")
    print(f"Elapsed time: {round(elapsed_time, 2)} seconds")
    print(f"CUDA Model size: {round(start_cuda_malloc / (1024 ** 3), 2)} GB")
    print(f"CUDA Memory Allocated: {round(cuda_malloc_usage, 2)} GB")
    print(f"CUDA Memory Inference: {round(cuda_mres_usage, 2)} GB")

#     images.append(image)
#     seeds.append(seed)

# print(seeds)
# image_grid(images, 1, samples, 512)